In [2]:
#coding=utf-8
import xlwings as xw
import re

# 打開活頁簿檔案
file_path = ('Han-Ggi.xlsx')
wb = xw.Book(file_path)

In [3]:
source_sheet = wb.sheets['jji-im-dui-jau']

end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

end_row = 4546


In [19]:
sheet = wb.sheets['jji-im-dui-jau']

# 轉換成十五音【聲母】
seng_boo_dict = {
    'ㄅ': 'p', 
    'ㄆ': 'ph', 
    'ㄇ': 'm', 
    'ㆠ': 'b', 
    'ㄉ': 't', 
    'ㄊ': 'th', 
    'ㄋ': 'n', 
    'ㄌ': 'l', 
    'ㄍ': 'k', 
    'ㄎ': 'kh', 
    'ㄏ': 'h', 
    'ㆣ': 'g', 
    'ㄗ': 'ch', 
    'ㄘ': 'chh', 
    'ㄙ': 's', 
    'ㆡ': 'j', 
    'ㄐ': 'ch', 
    'ㄑ': 'chh', 
    'ㄒ': 's', 
    'ㆢ': 'j', 
    'ㄇ': 'm', 
    'ㄣ': 'n', 
    'ㄚ': 'ng',  
}
seng_pattern = str.maketrans(seng_boo_dict)

# 轉換成十五音【韻母】
un_boo_dict = {
    'ㄚ': 'a', 
    'ㄧ': 'i', 
    'ㄨ': 'u', 
    'ㆤ': 'e', 
    'ㄛ': 'oo', 
    'ㄜ': 'o',
    'ㄞ': 'ai',
    'ㄠ': 'au', 
    '儿': 'o', 
    'ㆩ': 'ann', 
    'ㆪ': 'inn', 
    'ㆫ': 'unn', 
    'ㆥ': 'enn', 
    'ㆧ': 'oonn',         
    'ㄢ': 'an',
    'ㄤ': 'ang',
    'ㄣ': 'n',
    'ㄥ': 'ng', 
    'ㄇ': 'm',
    'ㆬ': 'm',
    'ㆭ': 'ng',
    'ㄅ': 'p', 
    'ㄉ': 't', 
    'ㄍ': 'k', 
    'ㄏ': 'h', 
}
un_pattern = str.maketrans(un_boo_dict)
    
row = 1
while row < end_row:
    # Read data from source_sheet 
    zu_im  = str(source_sheet.range('B' + str(row)).value)
    # 沒有注音，取下一個儲存格
    if zu_im.strip() == '':
        row += 1
        continue

    # 如果有聲母／韻母為濁音，則進行聲母／韻母符號更換
    zu_im_str = zu_im.replace('ㄝ', 'ㆤ') \
                .replace('ㄅ。', 'ㆠ') \
                .replace('ㄍ。', 'ㆣ') \
                .replace('ㄗ。', 'ㆡ') \
                .replace('ㄐ。', 'ㆢ') \
                .replace('ㄚ。', 'ㆩ') \
                .replace('ㄧ。', 'ㆪ') \
                .replace('ㄨ。', 'ㆫ') \
                .replace('ㆤ。', 'ㆥ') \
                .replace('ㄛ。', 'ㆧ')
    zu_im_chars = len(zu_im_str)
    
    # ===================================================
    # 處理「聲母」
    # ===================================================
    
    # 判斷：是否有聲母（若字串開頭為韻母，則無聲母）
    # pattern = re.compile(r'^[ㄚㆩㄧㆪㄨㆫㄝㆥㄛㆧㄜ儿]')
    pattern = re.compile(r'^[ㄚㆩㄧㆪㄨㆫㆤㆥㄛㆧㄜ儿ㄞㄠㄢㄤㄣㄥ]')
    seng_boo = ''
    un_boo = ''
    if not pattern.match(zu_im_str):
        seng_boo += zu_im_str[0] 
        un_boo = zu_im_str[1:zu_im_chars-1]
    else:
        un_boo = zu_im_str[0:zu_im_chars-1]
            
    # ===================================================
    # 處理「聲調」
    # ===================================================
    seng_tiau = zu_im_str[zu_im_chars-1]

    # 入聲韻尾使用「ㄅ、ㄉ、ㄍ、ㄏ」
    buffer = zu_im_str[zu_im_chars-2:]
    pattern2 = re.compile(r'[ㄅㄉㄍㄏ][-+]$')
    if pattern2.match(buffer):
        if seng_tiau == '-':
            seng_tiau_2 = 8
        elif seng_tiau == '+':
            seng_tiau_2 = 4
    else:
        if seng_tiau == '-':
            seng_tiau_2 = 1
        elif seng_tiau == '+':
            seng_tiau_2 = 7
        elif seng_tiau == 'ˇ':
            seng_tiau_2 = 3
        elif seng_tiau == 'ˊ':
            seng_tiau_2 = 5
        elif seng_tiau == 'ˋ':
            seng_tiau_2 = 2

    # 轉換成十五音
    seng = ''
    if seng_boo != '':
        seng += seng_boo_dict[seng_boo]
    
    un_boo_str = un_boo.replace('ㄛㄍ', 'ok') \
                    .replace('ㄛㄥ', 'ong') \
                    .replace('ㆤㄣ', 'an') \
                    .replace('ㄧㄛㄍ', 'iok') \
                    .replace('ㄧㆤㄉ', 'iat') \
                    .replace('ㄧㆤㄍ', 'ek') \
                    .replace('ㄧㆤㄥ', 'eng') \
                    .replace('ㄨㄚ', 'oa') \
                    .replace('ㄨㄚㄉ', 'oat') \
                    .replace('ㄨㆤ', 'oe') \
                    .replace('ㄨㄢ', 'oan')  

    # un = un_boo.translate(seng_pattern).translate(un_pattern)
    un = un_boo_str.translate(un_pattern)
    
    # Write to target cell
    sheet.range('C' + str(row)).value = seng_boo
    sheet.range('D' + str(row)).value = un_boo
    sheet.range('E' + str(row)).value = seng_tiau
    sheet.range('F' + str(row)).value = seng
    sheet.range('G' + str(row)).value = un
    sheet.range('H' + str(row)).value = seng_tiau_2
    row += 1
  